<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/bert_replace/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment Setup

In [1]:
!pip install keras_bert

In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

fatal: destination path 'bachelor_design' already exists and is not an empty directory.
/content/bachelor_design


In [3]:
!git checkout bert_replace
!git pull

Already on 'bert_replace'
Your branch is up to date with 'origin/bert_replace'.
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), done.
From https://github.com/supplient/bachelor_design
   ca2ea43..c3694a9  bert_replace -> origin/bert_replace
Updating ca2ea43..c3694a9
Fast-forward
 preprocess.py | 13 +++++++------
 1 file changed, 7 insertions(+), 6 deletions(-)


# Mount GDriver
Perpare Colab environment.

In [2]:
locale = False
try:
    import google.colab
    google.colab.drive.mount('/gdrive')
except ModuleNotFoundError:
    locale = True

In [3]:
addh = "/gdrive/My Drive"
if locale:
    addh = "/mnt/d/My Drive"
    print("[Locale] Using address head: " + addh)
else:
    print("[Colab] Using address head: " + addh)

[Locale] Using address head: /mnt/d/My Drive


# Prepare Input

In [1]:
import preprocess

Using TensorFlow backend.
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/supplient/.local/lib/python3.6/site-packages/tenso

In [4]:
SEQ_LEN = 512

train_char_seqs, train_tag_seqs = preprocess.load_file(
    addh + "/Graduation/Data/train_dict.txt")
test_char_seqs, test_tag_seqs = preprocess.load_file(
    addh + "/Graduation/Data/test_dict.txt")
train_token_id_seqs, train_segment_seqs, train_tag_id_seqs, tag_vocab = preprocess.preprocess(
    train_char_seqs, 
    train_tag_seqs,
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/vocab.txt",
    SEQ_LEN=SEQ_LEN
    )
test_token_id_seqs, test_segment_seqs, test_tag_id_seqs, tag_vocab = preprocess.preprocess(
    test_char_seqs, 
    test_tag_seqs,
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/vocab.txt",
    SEQ_LEN=512,
    tag_vocab=tag_vocab
    )

In [5]:
import pickle
with open(addh + "/Graduation/Model/tag_vocab.pkl", "wb") as fd:
    pickle.dump(tag_vocab, fd)

In [8]:
train_token_id_seqs[0]

[101,
 156,
 151,
 150,
 153,
 2675,
 7306,
 7318,
 3183,
 5759,
 4161,
 7901,
 8272,
 2810,
 7282,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0

In [9]:
train_segment_seqs[0]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [10]:
train_tag_id_seqs[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
tag_vocab

{'': 0,
 'B-EI': 6,
 'B-EIF': 10,
 'B-EO': 4,
 'B-EQ': 8,
 'B-ILF': 2,
 'I-EI': 7,
 'I-EIF': 11,
 'I-EO': 5,
 'I-EQ': 9,
 'I-ILF': 3,
 'O': 1}

# Build Model
using BERT

In [0]:
import keras
import keras_bert 

In [13]:
bert_model = keras_bert.load_trained_model_from_checkpoint(
    # config_file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_config.json", 
    # checkpoint file
    addh + "/Graduation/BERT/multi_cased_L-12_H-768_A-12/bert_model.ckpt", 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True

)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









In [0]:
input_token = keras.layers.Input(shape=(SEQ_LEN,))
input_segment = keras.layers.Input(shape=(SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

output = keras.layers.Dense(len(tag_vocab), activation=keras.activations.softmax)(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [15]:
model.compute_output_shape([(1, SEQ_LEN), (1, SEQ_LEN)])

(1, 512, 12)

In [16]:
bert_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [17]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
model_2 (Model)                 (None, 512, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 512, 12)      9228        model_2[1][0]              

# Fine-tuning

In [18]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.00004),
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"]
)

In [0]:
model.fit(
    x=[train_token_id_seqs, train_segment_seqs],
    y=[train_tag_id_seqs], 
    batch_size=8, 
    epochs=5,
    verbose=2,
    validation_data=([test_token_id_seqs, test_segment_seqs], [test_tag_id_seqs]),
    shuffle=True
)

Train on 4999 samples, validate on 2785 samples
Epoch 1/5
